In [1]:
import tensorflow as tf
import os
import sys


from preprocess import create_data_generators, get_class_map

print("TensorFlow version:", tf.__version__)




TensorFlow version: 2.20.0


In [2]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10

In [3]:

BASE_DIR = "../dataset" 
TRAIN_DIR = os.path.join(BASE_DIR, "train")
TEST_DIR = os.path.join(BASE_DIR, "test")

print(f"Train path exist: {os.path.exists(TRAIN_DIR)}")
print(f"Test path exist: {os.path.exists(TEST_DIR)}")

Train path exist: True
Test path exist: True


In [4]:
train_generator, test_generator = create_data_generators(
    TRAIN_DIR,
    TEST_DIR,
    IMG_SIZE,
    BATCH_SIZE
    )

labels = get_class_map(train_generator)

Found 2870 images belonging to 4 classes.
Found 394 images belonging to 4 classes.


In [5]:

X_batch, y_batch = next(train_generator) 
print("Batch image shape:", X_batch.shape)  

print("\nLoading Test Data:")

X_batch, y_batch = next(test_generator)  
print("Batch image shape:", X_batch.shape) 


Batch image shape: (32, 224, 224, 3)

Loading Test Data:
Batch image shape: (32, 224, 224, 3)


In [6]:
import tensorflow as tf

num_classes = len(labels)

model_baseline = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model_baseline.summary()

d:\SE\SP\SeniorProject\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,476 (42.61 MB)

 Trainable params: 11,169,476 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
model_baseline.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [8]:
import time
import math

steps_per_epoch = math.ceil(train_generator.samples / BATCH_SIZE)
validation_steps = math.ceil(test_generator.samples / BATCH_SIZE)

start = time.time()
history_baseline = model_baseline.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_generator,
    validation_steps=validation_steps,
    epochs=EPOCHS
)
train_time_sec = time.time() - start

print("Training time (sec):", round(train_time_sec, 2))

Epoch 1/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 205s 2s/step - accuracy: 0.4439 - loss: 1.2303 - val_accuracy: 0.2944 - val_loss: 2.1639
Epoch 2/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.5850 - loss: 0.9641 - val_accuracy: 0.3985 - val_loss: 2.4255
Epoch 3/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.6282 - loss: 0.8675 - val_accuracy: 0.3934 - val_loss: 2.2536
Epoch 4/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - accuracy: 0.6592 - loss: 0.7917 - val_accuracy: 0.3909 - val_loss: 3.1475
Epoch 5/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 137s 2s/step - accuracy: 0.6753 - loss: 0.7562 - val_accuracy: 0.3401 - val_loss: 5.0084
Epoch 6/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 138s 2s/step - accuracy: 0.6784 - loss: 0.7412 - val_accuracy: 0.3629 - val_loss: 2.6728
Epoch 7/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 138s 2s/step - accuracy: 0.6962 - loss: 0.7105 - val_accuracy: 0.4188 - val_loss: 2.9768
Epoch 8/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 138s 2s/step - accuracy: 0.7118 - loss: 0.6788 - val_accuracy: 0.4162 - v

In [9]:
baseline_loss, baseline_acc = model_baseline.evaluate(
    test_generator,
    steps=validation_steps
)
print("Baseline Loss:", baseline_loss)
print("Baseline Accuracy:", baseline_acc)

13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 267ms/step - accuracy: 0.4289 - loss: 2.8494
Baseline Loss: 2.8493857383728027
Baseline Accuracy: 0.4289340078830719


In [12]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# reset generator to start
test_generator.reset()

y_prob = model_baseline.predict(test_generator, steps=validation_steps)
y_pred = np.argmax(y_prob, axis=1)

y_true = test_generator.classes  # integer labels in directory order
class_names = list(test_generator.class_indices.keys())

cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)

print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_names))

13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 200ms/step
Confusion Matrix:
 [[12 13 62 13]
 [ 2 28 77  8]
 [ 3  2 98  2]
 [ 5  2 36 31]]

Classification Report:

                  precision    recall  f1-score   support

    glioma_tumor       0.55      0.12      0.20       100
meningioma_tumor       0.62      0.24      0.35       115
        no_tumor       0.36      0.93      0.52       105
 pituitary_tumor       0.57      0.42      0.48        74

        accuracy                           0.43       394
       macro avg       0.53      0.43      0.39       394
    weighted avg       0.52      0.43      0.38       394



In [ ]:
import os
os.makedirs("../models", exist_ok=True)

model_baseline.save("../models/baseline_cnn.keras")
print("Saved to ../models/baseline_cnn.keras")

Saved to ../models/baseline_cnn.keras
